In [2]:
# from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.vectorstores import DocArrayInMemorySearch
# from IPython.display import display, Markdown
# from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.indexes import VectorstoreIndexCreator
# from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
# from langchain.agents.agent_types import AgentType
# from langchain.memory import ConversationBufferMemory
# import tiktoken
# from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import (Language,RecursiveCharacterTextSplitter,)
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI

In [93]:
import os
# Prompt the user for their OpenAI API key
api_key = input("insert api key")
# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key
print("done!")

done!


In [4]:
context="""
push_to_confluence.py file is created to push the markdown format file into the confluence page.
Since we can't push markdown format file directly in to the confluence page we need to convert the markdown format file into html
then push it to confluence, we need to ask the User to input, confluence url, space key, username, api_token, then we will put those values into the 
cofluence function and push it to confluence by create an new page if it dosen't exist already, if it exists throw an error that it already exists. 
"""
code = """
import requests
import json
import base64
import markdown
import streamlit as st


def clear_submit():
    st.session_state["submit"] = True

# function
def confluence(confluence_url,space_key,username,api_token,page_title,mark_data):
    
    #reading file and string in into a variable
    # with open(uploaded_file.name, 'r') as file:
    #     mark_data = file.read()
        
    # Convert markdown to HTML
    html_text = markdown.markdown(mark_data)
    
    # Prepare data for the API request
    data = {
        "type": "page",
        "title": page_title,
        "space": {"key": space_key},
        "body": {
            "storage": {
                "value": html_text,
                "representation": "storage"
            }}}

    url = f"{confluence_url}/rest/api/content"
    # Send the POST request to create the page
    headers = {
    "Authorization": f"Basic {base64.b64encode(f'{username}:{api_token}'.encode()).decode()}",
    "X-Atlassian-Token": api_token,"Content-Type": "application/json"}


    response = requests.post(url, json=data, headers=headers)
    print(response.status_code)

    # Check for success
    if response.status_code == 200:
        st.success(f"{page_title} created successfully!", icon="✅")
        print("Page created successfully!")
    elif response.status_code == 400:
        st.error(f"{page_title} already exist:")
        print("Error creating page:", response.text)
    else:
        st.write("Error creating page:", response.text)
        print("Error creating page:", response.text)


# calling function with inputs
# with st.sidebar:
       
#     confluence_url = st.text_input("Confluence URL", )  
#     space_key = st.text_input("Space Key", )  
#     username = st.text_input("User Name", )   
#     api_token = st.text_input("API Key", ) 
#     page_title = st.text_input("Page Title", ) 
    # uploaded_file = st.file_uploader("Upload file",type=['txt','md'],help="Only txt,md  files are supported",)   
    
# if uploaded_file:
#     confluence(confluence_url,space_key,username,api_token,page_title,uploaded_file)
"""

In [9]:
# You can also see the separators used for a given language
# RecursiveCharacterTextSplitter
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=150, chunk_overlap=50
)
python_docs = python_splitter.create_documents([code])
python_docs


[Document(page_content='import requests\nimport json\nimport base64\nimport markdown\nimport streamlit as st'),
 Document(page_content='def clear_submit():\n    st.session_state["submit"] = True\n\n# function'),
 Document(page_content='def confluence(confluence_url,space_key,username,api_token,page_title,mark_data):\n    \n    #reading file and string in into a variable'),
 Document(page_content="#reading file and string in into a variable\n    # with open(uploaded_file.name, 'r') as file:\n    #     mark_data = file.read()"),
 Document(page_content='#     mark_data = file.read()\n        \n    # Convert markdown to HTML\n    html_text = markdown.markdown(mark_data)'),
 Document(page_content='html_text = markdown.markdown(mark_data)\n    \n    # Prepare data for the API request\n    data = {\n        "type": "page",'),
 Document(page_content='data = {\n        "type": "page",\n        "title": page_title,\n        "space": {"key": space_key},\n        "body": {\n            "storage": 

In [95]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_text(context)
print(texts)

['push_to_confluence.py file is created to push the markdown format file into the confluence page.', "Since we can't push markdown format file directly in to the confluence page we need to convert the", 'need to convert the markdown format file into html', 'then push it to confluence, we need to ask the User to input, confluence url, space key, username,', 'key, username, api_token, then we will put those values into the', "cofluence function and push it to confluence by create an new page if it dosen't exist already, if", 'exist already, if it exists throw an error that it already exists.']


In [96]:
# Create vector store
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_texts(texts=texts, embedding=embeddings)

In [97]:
print(vectorstore.search("what is this file about?", search_type='similarity'))

[Document(page_content='need to convert the markdown format file into html'), Document(page_content='push_to_confluence.py file is created to push the markdown format file into the confluence page.'), Document(page_content="Since we can't push markdown format file directly in to the confluence page we need to convert the"), Document(page_content='exist already, if it exists throw an error that it already exists.')]


In [118]:
# Create conversation chain
# llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo",max_tokens=7000)
llm = ChatOpenAI(openai_api_key=api_key, model ="gpt-3.5-turbo-0125", max_tokens=4000)

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided code:
<context>{context}</context>
Question: {input}
Code: {code}""")
document_chain = create_stuff_documents_chain(llm, prompt)
print(document_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['code', 'context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['code', 'context', 'input'], template='Answer the following question based only on the provided code:\n<context>{context}</context>\nQuestion: {input}\nCode: {code}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001B99119AF20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001B9911C8460>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=4000)
| StrOutputParser() config={'run_name': 'stuff_documents_chain'}


In [119]:
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
print(retriever)
print()
print(retrieval_chain)


tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B990D74790>

bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B990D74790>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['code', 'context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['code', 'context', 'input'], template='Answer the following question based only on the provided code:\n<context>{context}</context>\nQuestion: {input}\nCode: {code}'))])
            | ChatOpenAI(client=<o

In [122]:
inline_commenting_prompt = """1. Introduction

Purpose: Briefly describe the purpose of adding inline comments to the code. Emphasize the importance of making the code understandable and maintainable.
Scope of Code: Provide a brief overview of the functionality covered by the code that needs commenting.
2. Code Overview

Functionality: Describe the main functionality of the code, including any significant algorithms, data structures, or logic used.
Modules/Classes: List the major modules or classes included in the code and their roles.
3. Commenting Guidelines

Clarity and Brevity: Comments should be clear and concise, providing essential information without being verbose.
Relevance: Ensure that comments are relevant to the code they accompany and add value by explaining "why" something is done, not just "what" is done.
Consistency: Follow a consistent style for comments throughout the code to maintain readability.
Avoid Redundancy: Avoid stating the obvious that can be clearly understood from the code itself.
4. Specific Areas to Comment

Function Definitions: For each function, include comments that:
Describe the purpose of the function.
List the parameters and their types.
Explain the return value and type.
Complex Logic: Identify any complex or non-obvious logic sections and:
Explain the rationale behind the chosen approach.
Describe how the logic fits into the overall functionality.
Conditional Statements: For if, else, and switch statements, explain:
The conditions being checked.
Why these conditions are important.
Loops: For loops (for, while, do-while), comment on:
What the loop is achieving.
The termination condition of the loop.
Error Handling: Explain how errors are handled and:
What conditions are checked for errors.
How errors impact the flow of the program.
5. Examples

Example Comment for a Function: Provide a sample comment for a hypothetical or actual function to illustrate how to effectively comment functions.
Example Comment for Complex Logic: Provide an example of how to comment a complex piece of logic within the code.
6. Tools and Resources

Commenting Tools: Suggest any tools or plugins that can assist in generating or maintaining comments in the code.
Reference Materials: List any coding standards or style guides that provide additional guidelines on commenting practices.
7. Review Process

Peer Review: Encourage peer reviews focused on the quality and effectiveness of comments to ensure they add value and clarity.
Iterative Improvement: Emphasize the importance of continuously improving comments as the code evolves.
8. Conclusion

Summary: Recap the key points about the importance and methods of effective inline commenting.
Encouragement: Motivate the developer to maintain high standards in commenting to enhance code readability and maintainability.
"""

response = retrieval_chain.invoke({"input": f"""
Create the code documentation for the provided code in Markdown format// for a new commer to understand easily, elaborate every part of the code// start by writing title as code documentation
follow these rules: {inline_commenting_prompt}

""", "code": f"{code}"
})
print(response["answer"])

# Code Documentation

## Introduction
Purpose: The purpose of adding inline comments to the code is to make it easily understandable and maintainable. It helps new developers navigate the codebase more efficiently.

Scope of Code: The code provided is for converting a markdown file into HTML format and pushing it to a Confluence page. It involves reading the markdown file, converting it to HTML, and sending a POST request to create a Confluence page.

## Code Overview
Functionality: The main functionality of the code includes converting a markdown file to HTML and creating a Confluence page with the converted HTML content.

Modules/Classes: The code includes the `requests`, `json`, `base64`, `markdown`, and `streamlit` modules. It defines a function `clear_submit()` and a function `confluence()` for handling the conversion and pushing to Confluence.

## Commenting Guidelines
- Clarity and Brevity: Comments should be clear and concise.
- Relevance: Comments should explain the purpose an

In [128]:
response = retrieval_chain.invoke({"input": """
generate inline commnets for the following code and put those inline comments back to code // for a new commer to understand easily, elaborate every part of the code // return in the format 
code: ...
""", "code": f"{code}"
})
print(response["answer"])

```python
# Importing required libraries
import requests
import json
import base64
import markdown
import streamlit as st

# Function to set 'submit' session state value to True
def clear_submit():
    st.session_state["submit"] = True

# Function to push markdown content to Confluence
def confluence(confluence_url, space_key, username, api_token, page_title, mark_data):
    
    # Convert markdown content to HTML
    html_text = markdown.markdown(mark_data)
    
    # Prepare data for the API request
    data = {
        "type": "page",
        "title": page_title,
        "space": {"key": space_key},
        "body": {
            "storage": {
                "value": html_text,
                "representation": "storage"
            }}}
    
    # Define the API endpoint URL
    url = f"{confluence_url}/rest/api/content"
    
    # Set headers for the API request
    headers = {
        "Authorization": f"Basic {base64.b64encode(f'{username}:{api_token}'.encode()).decode()}",
       

In [125]:
code_quality_prompt = """Code Quality Document for [Project Name]

1. Introduction

Purpose of Document: Describe the purpose of this code quality document, including its intended audience.
Project Overview: Provide a brief overview of the project, including its objectives and scope.
2. Code Quality Objectives

Readability: Define the standards for how the code should be written to ensure it is easily understandable.
Maintainability: Outline the expectations for code to be easily maintained and updated.
Reliability: Describe the measures that will ensure the code performs its intended functions under specified conditions.
Efficiency: Set goals for the code to perform its functions in an efficient manner, optimizing for speed and resource usage.
Testability: Specify how the code should be structured to facilitate effective testing.
3. Coding Standards

Language-Specific Conventions: Detail the coding conventions for the programming languages used in the project.
Code Formatting: Outline the formatting standards (indentation, comments, naming conventions, etc.).
Documentation Standards: Describe the requirements for in-code documentation and external documentation.
4. Code Reviews

Review Process: Define the process for conducting code reviews, including roles and responsibilities.
Checklist: Include a checklist of common issues to look for during code reviews (e.g., security vulnerabilities, code smells).
Frequency and Timing: Specify how often code reviews should occur (e.g., after each major commit, before merging branches).
5. Testing Procedures

Unit Testing: Describe the approach for unit testing, including frameworks and tools to be used.
Integration Testing: Outline how and when integration testing should be performed.
Performance Testing: Detail the performance testing procedures to ensure efficiency standards are met.
Security Testing: Explain the security testing measures to ensure the application is secure from vulnerabilities.
6. Performance Optimization

Profiling: Discuss the use of profiling tools to identify performance bottlenecks.
Optimization Techniques: List common optimization techniques relevant to the project.
7. Tools and Technologies

Development Tools: List the development tools and IDEs recommended for the project.
Version Control: Specify the version control system and conventions for branch management.
Code Analysis Tools: Recommend tools for static code analysis, linters, and other quality assurance tools.
8. Exception Handling

Strategies: Define the strategies for handling exceptions and errors in the code.
Logging: Outline the logging practices to be followed for error monitoring and debugging.
9. Security Practices

Coding Practices: List secure coding practices to prevent common security issues like SQL injection, XSS, etc.
Code Access: Describe the security measures for accessing and storing the codebase.
10. Compliance and Standards

Industry Standards: Mention any industry-specific standards that need to be adhered to.
Legal Compliance: Outline any legal compliance issues related to the software (e.g., data protection laws).
11. Continuous Improvement

Feedback Mechanisms: Describe how feedback will be collected and used to improve code quality.
Quality Metrics: Define the metrics that will be used to measure code quality over time.
12. Conclusion

Summary: Recap the key points discussed in the document.
Future Considerations: Briefly discuss any potential future improvements or considerations for maintaining code quality.
Appendices

A: Glossary
B: Reference Documents
"""

response = retrieval_chain.invoke({"input": f"""
Check the code quality of the following code and provide suggestions if necessary// remember this is for new commers to understand // make it very comprehensive

""", "code": f"{code}"
})
print(response["answer"])

The code provided seems to be of good quality and well-structured. Here are a few suggestions for improvement:

1. Error Handling: The code handles the response status codes 200 and 400. It would be beneficial to add more detailed error handling for different status codes to provide more information to the user in case of any issues.

2. Documenting Functions: Adding docstrings to describe the functions, input parameters, and expected output would help new developers understand the purpose and functionality of each function.

3. Input Validation: Ensure that input validation is implemented to prevent errors or unexpected behavior. For example, checking if required input fields are filled before submitting the form.

4. Logging: Consider using a logging library to log errors, warnings, and information for better debugging and troubleshooting.

5. Commenting: While the code is well-commented, you could add more detailed comments to explain complex logic or decision points for better unde